In [2]:
import importlib
import business
importlib.reload(business)

<module 'business' from '/Users/a92461/Documents/sohan/messagecheck/rulescheck/business.py'>

In [5]:
# 第一轮规则审核代码 
import pandas as pd
import os
from datetime import datetime
import re
from typing import Tuple, Dict, List
from business import validate_business
from tqdm import tqdm

def first_round_rule_audit(input_file: str) -> pd.DataFrame:
    """
    执行第一轮规则审核并返回完整的审核结果DataFrame
    
    Args:
        input_file: 输入Excel文件路径
    
    Returns:
        df: 包含所有短信审核结果的DataFrame
    """
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    print(f"成功读取文件，共 {len(df)} 条记录")
    
    # 创建结果列
    df['总体操作类型'] = None
    df['业务操作类型'] = None
    
    # 审核计数器
    pass_count = 0
    fail_count = 0
    error_count = 0
    
    print("=============== 开始执行第一轮：规则审核==============")
    
    # 对每条短信执行规则审核
    for index, row in tqdm(df.iterrows(), total=len(df), desc="规则审核进度", ncols=100):
        
            
        try:
            # 调用业务规则审核
            business_passed, business_reason = validate_business(
                row['产品类型'],
                row['短信内容'], 
                row['短信签名'],
                row.get('账户类型')
            )
            
            # 更新结果
            if business_passed:
                df.loc[index, '总体操作类型'] = '放行'
                pass_count += 1
            else:
                df.loc[index, '总体操作类型'] = '失败'
                fail_count += 1
                
            df.loc[index, '业务操作类型'] = business_reason
            
        except Exception as e:
            print(f"处理行 {index} 时出错: {str(e)}")
            df.loc[index, '总体操作类型'] = '处理错误'
            df.loc[index, '业务操作类型'] = f"处理错误: {str(e)}"
            error_count += 1
    
    # 输出统计信息
    total = len(df)
    print(f"\n规则审核完成:")
    print(f"- 总记录数: {total}")
    print(f"- 放行数量: {pass_count} ({pass_count/total*100:.2f}%)")
    print(f"- 失败数量: {fail_count} ({fail_count/total*100:.2f}%)")
    if error_count > 0:
        print(f"- 错误数量: {error_count} ({error_count/total*100:.2f}%)")
    
    #可选：保存结果到Excel文件
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"123规则审核_{timestamp}.xlsx"
    df.to_excel(output_file, index=False)
    print(f"完整审核结果已保存至: {output_file}")
    
    
    # 统计不同情况的数量
    code_pass_human_fail = len(df[(df['总体操作类型'] == '放行') & (df['操作类型'] == '失败')])
    code_fail_human_pass = len(df[(df['总体操作类型'] == '失败') & (df['操作类型'] == '放行')])
    matched = len(df[df['总体操作类型'] == df['操作类型']])
    match_rate = (matched / total) * 100
    
    # 打印结果
    print('\n审核结果统计:')
    print(f'总样本数: {total}')
    print(f'代码放行但人工失败数量: {code_pass_human_fail}')
    print(f'代码失败但人工放行数量: {code_fail_human_pass}')
    print(f'匹配数量: {matched}')
    print(f'匹配率: {match_rate:.2f}%')


    return df

# 使用示例 
df_with_results = first_round_rule_audit("3月审核记录.xlsx")



读取文件: 3月审核记录.xlsx
成功读取文件，共 8133 条记录
=============== 开始执行第一轮：规则审核==============


规则审核进度:   0%|                                                        | 0/8133 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/g5/7ywkrjs52mn6f4r8ws5jsw8c0000gn/T/jieba.cache
Loading model cost 0.285 seconds.
Prefix dict has been built successfully.
规则审核进度: 100%|████████████████████████████████████████████| 8133/8133 [00:09<00:00, 871.85it/s]



规则审核完成:
- 总记录数: 8133
- 放行数量: 2869 (35.28%)
- 失败数量: 5264 (64.72%)
完整审核结果已保存至: 123规则审核_20250430_175550.xlsx

审核结果统计:
总样本数: 8133
代码放行但人工失败数量: 89
代码失败但人工放行数量: 4158
匹配数量: 3886
匹配率: 47.78%
